<a href="https://colab.research.google.com/github/Tretlaw/Machine-Learning-/blob/main/Cassava_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow_datasets as tfds
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load and Visualize dataset

In [4]:
datasets, info = tfds.load(name='cassava', with_info=True, as_supervised=True, split=['train', 'test', 'validation'])
info

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5656 [00:00<?, ? examples/s]

...

Shuffling /root/tensorflow_datasets/cassava/0.1.0.incompleteSLSQYX/cassava-test.tfrecord*...:   0%|          |…

Generating validation examples...:   0%|          | 0/1889 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cassava/0.1.0.incompleteSLSQYX/cassava-validation.tfrecord*...:   0%|     …

Dataset cassava downloaded and prepared to /root/tensorflow_datasets/cassava/0.1.0. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='cassava',
    full_name='cassava/0.1.0',
    description="""
    Cassava consists of leaf images for the cassava plant depicting healthy and
    four (4) disease conditions; Cassava Mosaic Disease (CMD), Cassava Bacterial
    Blight (CBB), Cassava Greem Mite (CGM) and Cassava Brown Streak Disease (CBSD).
    Dataset consists of a total of 9430 labelled images.
    The 9430 labelled images are split into a training set (5656), a test set(1885)
    and a validation set (1889). The number of images per class are unbalanced with
    the two disease classes CMD and CBSD having 72% of the images.
    """,
    homepage='https://www.kaggle.com/c/cassava-disease/overview',
    data_path=PosixGPath('/tmp/tmpv6tmfsz5tfds'),
    file_format=tfrecord,
    download_size=1.26 GiB,
    dataset_size=1.26 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': Cl

# Scale the data/Pixels

In [15]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.0
  return tf.image.resize(image, [224, 224]), tf.one_hot(label, 3)

# Batch the data

In [12]:
def get_dataset(batch_size=32):
  train_dataset_scaled = datasets[0].map(scale).shuffle(1000).batch(batch_size)
  test_dataset_scaled = datasets[1].map(scale).batch(batch_size)
  val_dataset_scaled = datasets[2].map(scale).batch(batch_size)
  return train_dataset_scaled, test_dataset_scaled, val_dataset_scaled

In [16]:
train_dataset, test_dataset, val_dataset = get_dataset()
train_dataset.cache()
val_dataset.cache

<bound method DatasetV2.cache of <_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>>

# Download pre-trained model

In [19]:
feature_extractor = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

In [20]:
feature_extractor_layer = hub.KerasLayer(feature_extractor, input_shape=(224, 224, 3))

In [21]:
feature_extractor_layer.trainable = False

In [23]:
model = tf.keras.Sequential([feature_extractor_layer, tf.keras.layers.Dropout(0.3),
                             tf.keras.layers.Dense(3, activation='softmax')])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dropout_1 (Dropout)         (None, 1280)              0         
                                                                 
 dense_1 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2261827 (8.63 MB)
Trainable params: 3843 (15.01 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


# Compile the model

In [24]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

# Train the model

In [ ]:
history = model.fit(train_dataset, epochs=20, validation_data=val_dataset)

Epoch 1/20
177/177 [==============================] - 308s 2s/step - loss: 0.4640 - accuracy: 0.3748 - val_loss: 0.3614 - val_accuracy: 0.3462
Epoch 2/20
177/177 [==============================] - 291s 2s/step - loss: 0.4000 - accuracy: 0.4006 - val_loss: 0.3316 - val_accuracy: 0.3584
Epoch 3/20
177/177 [==============================] - 306s 2s/step - loss: 0.3756 - accuracy: 0.4077 - val_loss: 0.3253 - val_accuracy: 0.3986
Epoch 4/20
177/177 [==============================] - 293s 2s/step - loss: 0.3635 - accuracy: 0.4088 - val_loss: 0.3187 - val_accuracy: 0.3896
Epoch 5/20
177/177 [==============================] - 284s 2s/step - loss: 0.3413 - accuracy: 0.4263 - val_loss: 0.3106 - val_accuracy: 0.3928
Epoch 6/20
177/177 [==============================] - 278s 2s/step - loss: 0.3469 - accuracy: 0.4144 - val_loss: 0.3039 - val_accuracy: 0.4055
Epoch 7/20
177/177 [==============================] - 298s 2s/step - loss: 0.3325 - accuracy: 0.4295 - val_loss: 0.3050 - val_accuracy: 0.4108